In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Formas de caracterizar en contexto

Haremos los siguientes experimentos para ver la mejor forma de caracterizar el contexto:
    - bolsa de palabras (unigramas, bigramas, subwords, con y sin stopwords)
    - Agregado por mi: tfidf con normalización para poner algunos pesos en las palabras
    - si hay interacción del usuario objetivo y cómo se representa
    - si hay respuestas
    - si hay multimedia y de qué tipo
__COMPLETAR__

Usaremos diferentes word embeddings, obtenidos de diferentes tipos de corpus: genéricos o específicos de lenguaje generado por usuario o específicos del corpus del que estamos aprendiendo.
    - Es decir, tres diferentes tipos de word embeddings:
      - SBWCE
      - con el mismo corpus de chats
      - con un corpus de twitter

## Cargando los datos de entrenamiento

In [7]:
import pandas as pd

In [8]:
df = pd.read_csv('short_ans3.csv')

Acordandonos de los datos..

In [4]:
sum(df.context.isnull())

1

In [5]:
df

,idchat,context,label
0,0,NaN,https://m.facebook.com/story.php?story_fbid=14...
1,0,https://m.facebook.com/story.php?story_fbid=14...,nonDef
2,1,😂😂😂😂😂😂😂😂😂😂. Que es este lugar 👀. Qur hace este...,Bienvenido
3,1,Bienvenido. Vengo a pasarle fotos obscenas e i...,👀
4,1,👀. El mundo de nunca jamas?),😂😂
5,1,😂😂,😂😂😂
6,1,😂😂😂,Oh sii
7,1,Oh sii. Seremos niños por siempre,Quienes ? 👀😂
8,1,Quienes ? 👀😂,Yo
9,1,Yo. 😏,😏


# Pre procesamiento

Como habíamos dicho es mejor pre procesar las palabras

In [4]:
from string import punctuation
import re
import unicodedata

In [5]:
STOP_WORDS = list(punctuation)

def remove_repeted(text):
    return re.sub(r'(\w)\1*', r'\1', text)

def remove_sw(text):
    return re.sub('{1}{0}{1}'.format(punctuation + '¡', '[]'), '', text)

def dig2num(word):
    return re.sub('(\d+[\WxX]*[^ ]*)+', 'NUM', word)

def remove_tildes(cadena):
    s = ''.join((c for c in unicodedata.normalize('NFD',cadena) if unicodedata.category(c) != 'Mn'))
    return s

def remove_ok(text):
    return re.sub('(ok)+[a-zA-Z]*', 'ok', text)

def remove_ja(text):
    return re.sub('(ja[ajs]*)+', 'ja', text)

def clean_text(text):
    return dig2num(remove_tildes(remove_ok(remove_ja(text.lower()))))

def clean_text_1(text):
    return remove_repeted(clean_text(remove_sw(text)))

In [9]:
df[1:].context.map(clean_text_1).head(30)

1         htpsmfacebokNUM hola te confundiste de numero
2     😂😂😂😂😂😂😂😂😂😂 que es este lugar 👀 qur hace este a...
3     bienvenido vengo a pasarle fotos obscenas e ir...
4                             👀 el mundo de nunca jamas
5                                                    😂😂
6                                                   😂😂😂
7                       oh si seremos ninos por siempre
8                                           quienes  👀😂
9                                                  yo 😏
10                                                    😏
11                                                    😏
12     😏 ja eso si que es bastante creativo muy bueno 😄
13                                                    😎
14                                                  🌚 🌚
15                                                    😏
16                                                 hola
17    hola 🇻🇪 que yo me desaparezca no hay lio pero ...
18    😱😱 quien ja mira que adulador es nuestro m

## Vectorizando

#### BOG: 1-gramas

In [62]:
vectorizer = CountVectorizer(min_df=10, preprocessor=clean_text_1, ngram_range=(1, 1))

In [63]:
representation = vectorizer.fit(df[1:].context)

Entonces obtenemos alrededor de tantas features:

In [172]:
print(representation.get_feature_names()[:100])
len(representation.get_feature_names())

['NUM', 'aNUM', 'abajo', 'abi', 'abierta', 'abiertas', 'abierto', 'abra', 'abran', 'abrazo', 'abrazos', 'abre', 'abren', 'abri', 'abril', 'abrir', 'abro', 'abuela', 'abuelo', 'aburido', 'abuso', 'ac', 'aca', 'acaba', 'acaban', 'acabar', 'acabaron', 'acabas', 'acabe', 'acabo', 'academia', 'acaso', 'aceder', 'acepta', 'acepte', 'acerca', 'aceso', 'acion', 'aclarar', 'aclaro', 'acompana', 'acompanar', 'acompano', 'acordaba', 'acordar', 'acordas', 'acordate', 'acorde', 'activa', 'activan', 'activar', 'active', 'actividad', 'actividades', 'activo', 'activos', 'acto', 'actores', 'actual', 'actualiza', 'actualizacion', 'actualizar', 'actualizo', 'acuerda', 'acuerdan', 'acuerdo', 'acv', 'ad', 'adelantar', 'adelante', 'adelanto', 'ademas', 'adentro', 'adios', 'adjuntado', 'admin', 'administrador', 'adri', 'adrian', 'aeropuerto', 'afecta', 'afiche', 'afuera', 'again', 'agara', 'agarar', 'agare', 'agaro', 'age', 'agenda', 'agendado', 'agendo', 'ago', 'agosto', 'agotada', 'agradece', 'agradeceria'

5204

In [176]:
BOG = representation.transform(df[1:].context)

In [179]:
print(BOG[:10])

  (0, 1213)	1
  (0, 2323)	1
  (0, 2354)	1
  (0, 3318)	1
  (0, 4638)	1
  (1, 275)	1
  (1, 1686)	1
  (1, 1772)	2
  (1, 2222)	1
  (1, 2840)	1
  (1, 4021)	1
  (1, 4075)	1
  (2, 487)	1
  (2, 2005)	1
  (2, 2512)	1
  (2, 3542)	1
  (2, 5013)	1
  (3, 1213)	1
  (3, 1540)	1
  (3, 2535)	1
  (3, 3185)	1
  (3, 3320)	1
  (6, 3261)	1
  (6, 3348)	1
  (6, 3802)	1
  (6, 4447)	1
  (6, 4449)	1
  (7, 4058)	1
  (8, 5193)	1


#### BOG: 2-gramas

In [11]:
representation.set_params(ngram_range=(1, 2))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(1, 2),
        preprocessor=<function clean_text at 0x7f0c0be776a8>,
        stop_words=['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [12]:
len(representation.fit(df[1:].context).get_feature_names())

14206

#### BOG: 3-gramas

In [76]:
representation.set_params(ngram_range=(1, 3))

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(1, 3),
        preprocessor=<function clean_text_1 at 0x7fd75c4a8510>,
        stop_words=None, strip_accents=None,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, vocabulary=None)

In [77]:
len(representation.fit(df[1:].context).get_feature_names())

16723

In [80]:
BOG_matrix = representation.transform(df[1:].context)

In [83]:
BOG_matrix.get_shape()

(56974, 16723)

In [84]:
df.context[28]

'chicos. una pregunta. cómo se escribe la dirección postal en vuestras ciudades?. es que los otros días vi una de francia y es un poco diferente a como yo escribo la mía. La de aca por donde vivo es 1010. El cod postal. así que si pongo n un sobre 1010 te llega?. Pero es por zona, no se si s igual por alla. qué raro sois los venezolanos. no?. No, obvio pones la direccion😂😂'

In [88]:
arr_repr = BOG_matrix[27].toarray()[0]
print('{ ', end='')
for i in BOG_matrix[27].indices:
    print('{0}: {1}'.format(representation.get_feature_names()[i], arr_repr[i]), end=', ')
print('}', end='')

{ NUM: 2, aca: 1, aca por: 1, ala: 1, asi: 1, asi que: 1, asi que si: 1, chicos: 1, chicos una: 1, ciudades: 1, cod: 1, como: 2, como se: 1, como yo: 1, de: 2, de aca: 1, dias: 1, diferente: 1, direcion: 2, donde: 1, en: 1, es: 4, es NUM: 1, es por: 1, es que: 1, es un: 1, es un poco: 1, escribe: 1, escribo: 1, igual: 1, igual por: 1, la: 4, la de: 1, la direcion: 2, la mia: 1, lega: 1, los: 2, los otros: 1, mia: 1, no: 3, no no: 1, no se: 1, no se si: 1, obvio: 1, otros: 1, pero: 1, pero es: 1, poco: 1, pones: 1, pongo: 1, por: 3, por ala: 1, por donde: 1, pregunta: 1, que: 3, que los: 1, que raro: 1, que si: 1, raro: 1, se: 2, se escribe: 1, se si: 1, si: 2, si igual: 1, sobre: 1, un: 2, un poco: 1, una: 2, una de: 1, una pregunta: 1, vi: 1, vivo: 1, yo: 1, zona: 1, }

#### BOG: (3,4)-Subwords

In [89]:
vectorizer = CountVectorizer(min_df=10, stop_words=STOP_WORDS, preprocessor=clean_text_1, analyzer='char', ngram_range=(3,4))

In [90]:
vectorizer

CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(3, 4),
        preprocessor=<function clean_text_1 at 0x7fd75c4a8510>,
        stop_words=['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~'],
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [91]:
representation = vectorizer.fit(df[1:].context)
BOG_matrix = representation.transform(df[1:].context)

In [92]:
len(representation.get_feature_names())

23187

In [84]:
df.context[28]

'chicos. una pregunta. cómo se escribe la dirección postal en vuestras ciudades?. es que los otros días vi una de francia y es un poco diferente a como yo escribo la mía. La de aca por donde vivo es 1010. El cod postal. así que si pongo n un sobre 1010 te llega?. Pero es por zona, no se si s igual por alla. qué raro sois los venezolanos. no?. No, obvio pones la direccion😂😂'

In [93]:
arr_repr = BOG_matrix[27].toarray()[0]
for i in BOG_matrix[27].indices:
    print('{0}: {1}'.format(representation.get_feature_names()[i], arr_repr[i]), end=', ')

 NU: 2,  NUM: 2,  a : 1,  a c: 1,  ac: 1,  aca: 1,  al: 1,  ala: 1,  as: 1,  asi: 1,  ci: 1,  ciu: 1,  co: 3,  cod: 1,  com: 2,  de: 2,  de : 2,  di: 4,  dia: 1,  dif: 1,  dir: 2,  do: 1,  don: 1,  en: 1,  en : 1,  es: 6,  es : 4,  esc: 2,  fr: 1,  fra: 1,  ig: 1,  igu: 1,  la: 4,  la : 4,  le: 1,  leg: 1,  lo: 2,  los: 2,  mi: 1,  mia: 1,  n : 1,  no: 3,  no : 3,  ob: 1,  obv: 1,  ot: 1,  otr: 1,  pe: 1,  per: 1,  po: 8,  poc: 1,  pon: 2,  por: 3,  pos: 2,  pr: 1,  pre: 1,  qu: 3,  que: 3,  ra: 1,  rar: 1,  s : 1,  se: 2,  se : 2,  si: 2,  si : 2,  so: 2,  sob: 1,  un: 4,  un : 2,  una: 2,  ve: 1,  ven: 1,  vi: 2,  vi : 1,  viv: 1,  vu: 1,  vue: 1,  y : 1,  y e: 1,  yo: 1,  yo : 1,  zo: 1,  zon: 1, M c: 1, M co: 1, M l: 1, M le: 1, NUM: 2, NUM : 2, UM : 2, UM c: 1, UM l: 1, a c: 2, a co: 2, a d: 4, a de: 2, a di: 2, a l: 1, a la: 1, a m: 1, a mi: 1, a n: 1, a no: 1, a p: 3, a pe: 1, a po: 1, a pr: 1, a q: 1, a qu: 1, a y: 1, a y : 1, aca: 1, aca : 1, ade: 1, ades: 1, al : 3, al a: 1, 

### TFidf: 1-gramas

In [224]:
vectorizer = TfidfVectorizer(min_df=10, preprocessor=clean_text_1, ngram_range=(1, 1), norm='l2')

In [225]:
representation = vectorizer.fit(df[1:].context)

In [226]:
BOG_matrix = representation.transform(df[1:].context)

Entonces podemos decir que la frase:

In [227]:
df.context[7]

'Oh sii. Seremos niños por siempre'

Esta representada como:

In [228]:
print(BOG_matrix[6])

  (0, 4449)	0.448864432483
  (0, 4447)	0.234091170894
  (0, 3802)	0.288960119172
  (0, 3348)	0.47924659666
  (0, 3261)	0.656160646548


Donde los features son...

In [232]:
representation.get_feature_names()[4449], representation.get_feature_names()[4447], representation.get_feature_names()[3802], representation.get_feature_names()[3348], representation.get_feature_names()[3261]

('siempre', 'si', 'por', 'oh', 'ninos')

### TFidf: 3-gramas

In [95]:
vectorizer = TfidfVectorizer(min_df=10, preprocessor=clean_text_1, ngram_range=(1, 3), norm='l2')

In [96]:
representation = vectorizer.fit(df[1:].context)

In [97]:
BOG_matrix = representation.transform(df[1:].context)

In [13]:
print(BOG_matrix[:10])

  (0, 14681)	0.255535071528
  (0, 10673)	0.53663367644
  (0, 6797)	0.697638942057
  (0, 6656)	0.352211692948
  (0, 2883)	0.189665374821
  (1, 13190)	0.443227352735
  (1, 12594)	0.260644641436
  (1, 12496)	0.113825661263
  (1, 8792)	0.312539115049
  (1, 6318)	0.237512204515
  (1, 5451)	0.483620546024
  (1, 4910)	0.44617459802
  (1, 4852)	0.1453280125
  (1, 931)	0.338354876003
  (2, 16018)	0.455086219987
  (2, 11418)	0.522142267358
  (2, 7063)	0.462061312057
  (2, 5900)	0.360124521208
  (2, 1382)	0.420801209766
  (3, 10677)	0.390404472585
  (3, 9956)	0.473677665079
  (3, 7382)	0.547445245685
  (3, 4201)	0.515860506833
  (3, 3969)	0.175011502472
  (3, 2883)	0.163635056944
  (6, 14235)	0.359178198001
  (6, 13969)	0.187318127356
  (6, 11975)	0.231223878274
  (6, 10727)	0.599742517453
  (6, 10721)	0.383489794533
  (6, 10150)	0.525055187203
  (7, 13142)	1.0
  (8, 16554)	1.0


Entonces podemos decir que la frase:

In [14]:
df.context[28]

'chicos. una pregunta. cómo se escribe la dirección postal en vuestras ciudades?. es que los otros días vi una de francia y es un poco diferente a como yo escribo la mía. La de aca por donde vivo es 1010. El cod postal. así que si pongo n un sobre 1010 te llega?. Pero es por zona, no se si s igual por alla. qué raro sois los venezolanos. no?. No, obvio pones la direccion😂😂'

Esta representada como:

In [15]:
print(BOG_matrix[27])

  (0, 16720)	0.124010228538
  (0, 16554)	0.0504677776659
  (0, 16245)	0.118677121056
  (0, 16134)	0.0873732335623
  (0, 15721)	0.113909106994
  (0, 15656)	0.111706514749
  (0, 15622)	0.109525659716
  (0, 15574)	0.0921570206326
  (0, 15481)	0.100784558462
  (0, 14325)	0.0928691100846
  (0, 14067)	0.127385614469
  (0, 13969)	0.0865077404921
  (0, 13751)	0.0919649009802
  (0, 13644)	0.139280590804
  (0, 13612)	0.0962287447121
  (0, 13212)	0.101262869165
  (0, 12915)	0.0891770737975
  (0, 12879)	0.123415330844
  (0, 12744)	0.105030590194
  (0, 12496)	0.108631627268
  (0, 12183)	0.0967615812843
  (0, 11993)	0.116665793098
  (0, 11984)	0.141938329461
  (0, 11975)	0.160176611359
  (0, 11964)	0.0982929802144
  :	:
  (0, 4997)	0.0841566810893
  (0, 4989)	0.113909106994
  (0, 4853)	0.114719000573
  (0, 4852)	0.184928639234
  (0, 4392)	0.0435597008661
  (0, 3822)	0.076456038442
  (0, 3754)	0.244042210164
  (0, 3677)	0.128973205425
  (0, 3626)	0.0943999070126
  (0, 2897)	0.120258907758
  (0, 2883)

Donde los features son...

In [100]:
dict_repr = dict()
arr_repr = BOG_matrix[27].toarray()[0]
for i in BOG_matrix[27].indices:
    print('{0}: {1}'.format(representation.get_feature_names()[i], arr_repr[i]), end=', ')
    dict_repr[representation.get_feature_names()[i]] = arr_repr[i]

zona: 0.12401022853833427, yo: 0.05046777766587819, vivo: 0.11867712105649862, vi: 0.08737323356228918, una pregunta: 0.11390910699436023, una de: 0.11170651474885525, una: 0.10952565971561047, un poco: 0.09215702063255835, un: 0.10078455846229553, sobre: 0.09286911008457037, si igual: 0.1273856144693643, si: 0.08650774049210937, se si: 0.09196490098019389, se escribe: 0.1392805908040324, se: 0.09622874471212998, raro: 0.1012628691652938, que si: 0.08917707379754441, que raro: 0.12341533084436813, que los: 0.10503059019403427, que: 0.10863162726785722, pregunta: 0.09676158128433333, por donde: 0.11666579309801224, por ala: 0.14193832946119114, por: 0.16017661135873373, pongo: 0.09829298021440097, pones: 0.1153989093181467, poco: 0.08583486140073365, pero es: 0.10350817183046465, pero: 0.0522557791260144, otros: 0.09800815755778436, obvio: 0.09930259913740713, no se si: 0.09278835045911256, no se: 0.06596199382689194, no no: 0.09431076882268218, no: 0.11333417403034404, mia: 0.111301283

Palabras más representativas para la frase:

In [103]:
sorted(dict_repr.items(), key=lambda x: x[1], reverse=True)[:10]

[('la direcion', 0.2644332485458174),
 ('direcion', 0.24404221016411312),
 ('es', 0.18492863923364514),
 ('por', 0.16017661135873373),
 ('la', 0.15823958720210332),
 ('aca por', 0.14514956217267053),
 ('ciudades', 0.14514956217267053),
 ('chicos una', 0.14193832946119114),
 ('por ala', 0.14193832946119114),
 ('es un poco', 0.14011676379920279)]

## Word Embeddings

Tomamos algunos embeddings de:
- [Glove](https://github.com/stanfordnlp/GloVe)
- [wordvector](https://github.com/Kyubyong/wordvectors)?
- [SBWCE](http://crscardellino.me/SBWCE/)
     